<a href="https://colab.research.google.com/github/Alagha1/DSA-Machine-Learning-Project/blob/main/Liver_Fibrosis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
from google.colab import files
HCV = files.upload()

Saving HCV-Egy-Data.csv to HCV-Egy-Data.csv


In [4]:
import pandas as pd
pd.set_option('display.max_columns', None)

Liver = pd.read_csv('HCV-Egy-Data.csv')

print(Liver.head())

   Age   Gender  BMI  Fever  Nausea/Vomting  Headache   Diarrhea   \
0    56       1   35      2               1          1          1   
1    46       1   29      1               2          2          1   
2    57       1   33      2               2          2          2   
3    49       2   33      1               2          1          2   
4    59       1   32      1               1          2          1   

   Fatigue & generalized bone ache   Jaundice   Epigastric pain     WBC  \
0                                 2          2                 2   7425   
1                                 2          2                 1  12101   
2                                 1          1                 1   4178   
3                                 1          2                 1   6490   
4                                 2          2                 2   3661   

       RBC  HGB    Plat  AST 1  ALT 1  ALT4  ALT 12  ALT 24  ALT 36  ALT 48  \
0  4248807   14  112132     99     84    52     109    

In [5]:
#Determining the number observations and features
print('The number of observations in my data is', Liver.shape[0])
print('The number of features in my data is', Liver.shape[1])

#Checking for missing values
print('The number of missing values in my data is', Liver.isna().sum().sum())

The number of observations in my data is 1385
The number of features in my data is 29
The number of missing values in my data is 0


In [6]:
#Data Wrangling
#Import libraries and packages
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_curve, auc, confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate


In [7]:
#Quantitative variables should be placed into discrete categories per the dataset authors.

#Age categories: [0; 32], ]32; 37], ]37; 42],]42; 47], ]47; 52], ]52; 57],]57; 62]

Liver.loc[Liver['Age '] <=32, 'Age_C'] = 0
Liver.loc[(Liver['Age '] > 32) & (Liver['Age '] <= 37), 'Age_C'] = 1
Liver.loc[(Liver['Age '] > 37) & (Liver['Age '] <= 42), 'Age_C'] = 2
Liver.loc[(Liver['Age '] > 42) & (Liver['Age '] <= 47), 'Age_C'] = 3
Liver.loc[(Liver['Age '] > 47) & (Liver['Age '] <= 52), 'Age_C'] = 4
Liver.loc[(Liver['Age '] > 52) & (Liver['Age '] <= 57), 'Age_C'] = 5
Liver.loc[(Liver['Age '] > 57) & (Liver['Age '] <= 62), 'Age_C'] = 6

#print(pd.crosstab(Liver['Age '],Liver['Age_C'], dropna = False))

#BMI categories: [0; 18:5[ [18:5; 25[, [25; 30[, [30; 35[, [35; 40[

Liver.loc[Liver['BMI'] < 18.5, 'BMI_C'] = 0
Liver.loc[(Liver['BMI'] >= 18.5) & (Liver['BMI'] < 25), 'BMI_C'] = 1
Liver.loc[(Liver['BMI'] >= 25) & (Liver['BMI'] < 30), 'BMI_C'] = 2
Liver.loc[(Liver['BMI'] >= 30) & (Liver['BMI'] < 35), 'BMI_C'] = 3
Liver.loc[(Liver['BMI'] >= 35) & (Liver['BMI'] < 40), 'BMI_C'] = 4
Liver.loc[Liver['BMI'] >= 40, 'BMI_C'] = 5

#print(pd.crosstab(Liver['BMI'], Liver['BMI_C'], dropna=False))

#WBC categories: [0; 4000[, [4000; 11000[, [11000; 12101]
Liver.loc[Liver['WBC'] < 4000, 'WBC_C'] = 0
Liver.loc[(Liver['WBC'] >= 4000) & (Liver['WBC'] < 11000), 'WBC_C'] = 1
Liver.loc[(Liver['WBC'] >= 11000) & (Liver['WBC'] <= 12101), 'WBC_C'] = 2

#print(pd.crosstab(Liver['WBC'], Liver['WBC_C'], dropna = False))

#RBC categories: 0; 3000000[, [3000000; 5000000[,[5000000; 5018451]
Liver.loc[Liver['RBC'] < 3000000, 'RBC_C'] = 0
Liver.loc[(Liver['RBC'] >= 3000000) & (Liver['RBC'] < 5000000), 'RBC_C'] = 1
Liver.loc[(Liver['RBC'] >= 5000000) & (Liver['RBC'] <= 5018451), 'RBC_C'] = 2

#HGB categories: If (Gender = 1) <14, >= 14& <17.5, > 17.5If (Gender = 2) <12.3, >= 12.3& <15.3, >= 15.3
Liver.loc[(Liver['Gender'] == 1) & (Liver['HGB'] < 14), 'HGB_C'] = 0
Liver.loc[(Liver['Gender'] == 1) & (Liver['HGB'] >= 14) & (Liver['HGB'] < 17.5), 'HGB_C'] = 1
Liver.loc[(Liver['Gender'] == 1) & (Liver['HGB'] > 17.5), 'HGB_C'] = 2

Liver.loc[(Liver['Gender'] == 2) & (Liver['HGB'] < 12.3), 'HGB_C'] = 0
Liver.loc[(Liver['Gender'] == 2) & (Liver['HGB'] >= 12.3) & (Liver['HGB'] < 15.3), 'HGB_C'] = 1
Liver.loc[(Liver['Gender'] == 2) & (Liver['HGB'] > 15.3), 'HGB_C'] = 2

#print(pd.crosstab(Liver['RBC'], Liver['RBC_C'], dropna = False))

#Platelet categories: [93013; 100000[, [100000; 255000[,[255000; 226465[
Liver.loc[(Liver['Plat'] >= 93013) & (Liver['Plat'] < 100000), 'PLT_C'] = 0
Liver.loc[(Liver['Plat'] >= 100000) & (Liver['Plat'] < 255000), 'PLT_C'] = 1
Liver.loc[(Liver['Plat'] >= 255000) & (Liver['Plat'] <= 226465), 'PLT_C'] = 2

#print(pd.crosstab(Liver['Plat'], Liver['PLT_C'], dropna = False))

#AST 1, ALT 1, ALT 4, ALT 12, ALT 24, ALT 36, ALT 48, ALT after 24 w categories: [0; 20[, [20; 40], ]40; 128]
Liver.loc[(Liver['AST 1'] >= 0) & (Liver['AST 1'] < 20), 'AST_1_C'] = 0
Liver.loc[(Liver['AST 1'] >= 20) & (Liver['AST 1'] < 40), 'AST_1_C'] = 1
Liver.loc[(Liver['AST 1'] >= 40) & (Liver['AST 1'] <= 128), 'AST_1_C'] = 2

Liver.loc[(Liver['ALT 1'] >= 0) & (Liver['ALT 1'] < 20), 'ALT_1_C'] = 0
Liver.loc[(Liver['ALT 1'] >= 20) & (Liver['ALT 1'] < 40), 'ALT_1_C'] = 1
Liver.loc[(Liver['ALT 1'] >= 40) & (Liver['ALT 1'] <= 128), 'ALT_1_C'] = 2

Liver.loc[(Liver['ALT4'] >= 0) & (Liver['ALT4'] < 20), 'ALT_4_C'] = 0
Liver.loc[(Liver['ALT4'] >= 20) & (Liver['ALT4'] < 40), 'ALT_4_C'] = 1
Liver.loc[(Liver['ALT4'] >= 40) & (Liver['ALT4'] <= 128), 'ALT_4_C'] = 2

Liver.loc[(Liver['ALT 12'] >= 0) & (Liver['ALT 12'] < 20), 'ALT_12_C'] = 0
Liver.loc[(Liver['ALT 12'] >= 20) & (Liver['ALT 12'] < 40), 'ALT_12_C'] = 1
Liver.loc[(Liver['ALT 12'] >= 40) & (Liver['ALT 12'] <= 128), 'ALT_12_C'] = 2

Liver.loc[(Liver['ALT 24'] >= 0) & (Liver['ALT 24'] < 20), 'ALT_24_C'] = 0
Liver.loc[(Liver['ALT 24'] >= 20) & (Liver['ALT 24'] < 40), 'ALT_24_C'] = 1
Liver.loc[(Liver['ALT 24'] >= 40) & (Liver['ALT 24'] <= 128), 'ALT_24_C'] = 2

Liver.loc[(Liver['ALT 36'] >= 0) & (Liver['ALT 36'] < 20), 'ALT_36_C'] = 0
Liver.loc[(Liver['ALT 36'] >= 20) & (Liver['ALT 36'] < 40), 'ALT_36_C'] = 1
Liver.loc[(Liver['ALT 36'] >= 40) & (Liver['ALT 36'] <= 128), 'ALT_36_C'] = 2

Liver.loc[(Liver['ALT 48'] >= 0) & (Liver['ALT 48'] < 20), 'ALT_48_C'] = 0
Liver.loc[(Liver['ALT 48'] >= 20) & (Liver['ALT 48'] < 40), 'ALT_48_C'] = 1
Liver.loc[(Liver['ALT 48'] >= 40) & (Liver['ALT 48'] <= 128), 'ALT_48_C'] = 2

Liver.loc[(Liver['ALT after 24 w'] >= 0) & (Liver['ALT after 24 w'] < 20), 'ALT after 24 w_C'] = 0
Liver.loc[(Liver['ALT after 24 w'] >= 20) & (Liver['ALT after 24 w'] < 40), 'ALT after 24 w_C'] = 1
Liver.loc[(Liver['ALT after 24 w'] >= 40) & (Liver['ALT after 24 w'] <= 128), 'ALT after 24 w_C'] = 2

#print(pd.crosstab(Liver['AST 1'], Liver['AST_1_C'], dropna = False))

#RNA Categories <=5, > 5
Liver.loc[Liver['RNA Base'] <= 5, 'RNA Base_C'] = 0
Liver.loc[Liver['RNA Base'] > 5, 'RNA Base_C'] = 1

Liver.loc[Liver['RNA 4'] <= 5, 'RNA 4_C'] = 0
Liver.loc[Liver['RNA 4'] > 5, 'RNA 4_C'] = 1

Liver.loc[Liver['RNA 12'] <= 5, 'RNA 12_C'] = 0
Liver.loc[Liver['RNA 12'] > 5, 'RNA 12_C'] = 1

Liver.loc[Liver['RNA EOT'] <= 5, 'RNA EOT_C'] = 0
Liver.loc[Liver['RNA EOT'] > 5, 'RNA EOT_C'] = 1

Liver.loc[Liver['RNA EF'] <= 5, 'RNA EF_C'] = 0
Liver.loc[Liver['RNA EF'] > 5, 'RNA EF_C'] = 1


#Categegorizing the target feature as 0 (cirrhosis absent) and 1 (cirrhosis present)
Liver.loc[Liver['Baselinehistological staging'] ==4, 'Cirrhosis'] = 1
Liver.loc[Liver['Baselinehistological staging'] !=4, 'Cirrhosis'] = 0

print(pd.crosstab(Liver['Baselinehistological staging'], Liver['Cirrhosis'], dropna = False))
#print(Liver['Baselinehistological staging'].value_counts())

Cirrhosis                     0.0  1.0
Baselinehistological staging          
1                             336    0
2                             332    0
3                             355    0
4                               0  362


In [8]:
#Modeling -  build ML model.

#Split the data into target and feature 

y = Liver['Cirrhosis']

X_1 = Liver[['Age_C', 'Gender', 'BMI_C', 'WBC_C', 'RBC_C', 'HGB_C', 'PLT_C', 'Fever', 'Jaundice ', 'Nausea/Vomting','Diarrhea ', 'Fatigue & generalized bone ache ', 'Headache ',   
             'Epigastric pain ']]


X_2 = Liver[['Age_C', 'Gender', 'BMI_C', 'WBC_C', 'RBC_C', 'HGB_C', 'PLT_C', 'Fever', 'Jaundice ', 'Nausea/Vomting','Diarrhea ', 'Fatigue & generalized bone ache ', 'Headache ',   
             'Epigastric pain ','AST_1_C','ALT_1_C', 'ALT_4_C', 'ALT_12_C','ALT_24_C' ,'ALT_36_C', 'ALT_48_C', 'ALT after 24 w_C', 'RNA Base_C', 'RNA 4_C', 'RNA 12_C', 'RNA EOT_C', 'RNA EF_C']]


#Train-test split#

X_1_train, X_1_test, y_train, y_test = train_test_split(X_1, y, test_size=0.25, random_state=42)
X_2_train, X_2_test, y_train, y_test = train_test_split(X_2, y, test_size=0.25, random_state=42)


In [9]:
#Fit the logistic regression model to the training data without the specialized lab values

model_1 = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000)

model_1.fit(X_1_train, y_train)

#Fit the logistic regression model to the training data with the specialized lab values.  At first I got an
#error message about the model not converging.  I googled it and found that one solution was to increase
#the maximum number of iterations.  I did that, and now the model works.

model_2 = LogisticRegression(random_state=0, solver='lbfgs', max_iter=1000)

model_2.fit(X_2_train, y_train)

LogisticRegression(max_iter=1000, random_state=0)

In [10]:
#Evaluation

#Calculate 10-fold CV accuracy for the model without the specialized lab values

scores_1 = cross_val_score(model_1, X_1_train, y_train, cv=10)

print('The accuracy of Model 1 is', scores_1.mean())

#The 10-fold CV accuracy without the extra lab values is about 0.74



#Calculate the AUC for the model without the specialized lab values

probs_1 = pd.DataFrame(model_1.predict_proba(X_1_train))
fpr_1_, tpr_1_, thresholds_ = roc_curve(y_train, probs_1[1])
print('The AUC for Model 1 is', auc(fpr_1_, tpr_1_))

#The AUC for the model without the extra lab values is about 0.56


scores_2 = cross_val_score(model_2, X_2_train, y_train, cv=10)

print('The accuracy of Model 2 is', scores_2.mean())

#The 10-fold CV accuracy with the extra lab values is just about the same 0.74

probs_2 = pd.DataFrame(model_2.predict_proba(X_2_train))
fpr_2_, tpr_2_, thresholds_ = roc_curve(y_train, probs_2[1])
print('The AUC of Model 2 is', auc(fpr_2_, tpr_2_))

#The AUC for the model with the extra lab values is about 0.57

The accuracy of Model 1 is 0.742783793876027
The AUC for Model 1 is 0.5587349470749112
The accuracy of Model 2 is 0.7418222554144884
The AUC of Model 2 is 0.572254040426121


In [11]:
#Because the model results are so close, let's evaluate both of them using the test set

print(model_1.score(X_1_test, y_test))
print(model_2.score(X_2_test, y_test))

#The testing accuracy is the same for both

0.7262247838616714
0.7262247838616714


In [ ]:
#Results
#Based on the results (accuracy and AUC) from my logistic regression model, there is no evidence that adding in the new lab values would aid physicians in the diagnosis of liver cirrhosis in these patients.
#It would be better to use save this money and use it for more affective diagnostic tests or treatments.

#Reflection
#I really didn't like doing all the data wrangling. That was tedious.
#I thought it was interesting that adding the new lab values didn't add predictive ability to the mdoel. I thought that it would.
#If I had more time, I would like to try using other classification models like KNN or tree-based models to see if my results might be different using one of those, especially because the accuracy of my model wasn't very good.

